In [149]:
'''
Return variables dictionary from JSON url. 
'''
def read_census_json(json_url): 
    
    # Make a GET request to the JSON url 
    response = requests.get(json_url)
    
    # Check if it was successful (status code == 200)
    if response.status_code == 200: 
        
        # This is the initial response, in dictionary form 
        json_dict = response.json()
        
        # There is only one key, so reassign the dictionary variable to that key 
        variables_dict = json_dict['variables']
        print("JSON dictionary returned")
    
    else: 
        print("Couldn't read JSON")
    
    return variables_dict

In [150]:
'''
Parse the dictionary so that it is intelligible. 
This includes all concepts, and will be filtered by an input group code or concept in the next function.
'''
def return_group_dictionary(json_dictionary): 
    
    # Initialize count 
    count = 0 
    
    # Initialize dictionaries 
    group_dictionary            = {} 
    
    # Iterate through JSON dictionary 
    for code, values in json_dictionary.items(): 
        
        try:
            # Identify group variable 
            group   = values['group']
            
            # Initialize dictionary key for group 
            if group not in group_dictionary.keys() and "," not in group: 
                group_dictionary[group]            = {} 
                group_dictionary[group]['concept'] = []
                group_dictionary[group]['codes']   = []
            else: 
                pass
            
        except: 
            pass 
        
        try: 
            # Identify concept variable 
            concept = values['concept']
            
            # Initialize dictionary key for group 
            group_dictionary[group]['concept'] = concept
            group_dictionary[group]['codes'].append(code)

        except KeyError:
            pass
        
    if group_dictionary != {}: 
        print('Group dictionary returned')
    else: 
        print('Group dictionary empty')
        
    return group_dictionary

In [151]:
'''
This uses the block and filter words to refine the dictionary created in the previous function and return it as a dataframe of concepts and codes.
'''
def print_group_dictionary(group_dictionary, filter_words, block_words, print_groups, print_codes): 
    
    # Initialize empty lists 
    group_codes = []
    concepts    = []
    
    # For group code, concept/codes dictionary
    for group_code, concept_codes_dict in group_dictionary.items(): 
        go = False 
        
        # Activate by filter word 
        for filter_word in filter_words: 
            if filter_word in concept_codes_dict['concept'] or filter_words == []: 
                go = True
        
        # Block by block word
        for block_word in block_words: 
            if block_word in concept_codes_dict['concept']:
                go = False
            
        while go == True:
            
            # Print if specified 
            if print_groups == True: 
                print("---------------")
                print(f"Group Code: {group_code}")
                print("---------------")
                
                print("Concept: ")
                print(" > ", concept_codes_dict['concept'])
                print("")

            if print_codes == True: 
                print("Codes: ")
                for code in concept_codes_dict['codes']: 
                    print(" >> ", code)
            
            # Append values to lists for dataframe 
            group_codes.append(group_code)
            concepts.append(concept_codes_dict['concept'])
                
            go = False 
    
    # Construct dataframe 
    acs_dataframe = pd.DataFrame({'Group Codes':group_codes, 'Concepts':concepts})
    
    print('Group dictionary filtered and passed to dataframe')
            
    return acs_dataframe

In [152]:
'''
Return a list of variable codes based on a designated group code. 
'''
def find_variable_codes(input_group_code, group_dictionary): 
    
    # For group code, concept/codes dictionary
    for group_code, concept_codes_dict in group_dictionary.items(): 
        if group_code == input_group_code: 
            variable_codes = concept_codes_dict['codes']
            
    return variable_codes 

In [153]:
'''
This creates a dictionary of codes and their values. 
'''
def create_code_label_dictionary(variable_codes, json_dictionary): 
    
    code_label_dictionary = {} 
    code_label_dictionary['B01001_001E'] = 'Total Population'

    for code in variable_codes: 
        label = json_dictionary[code]['label'].split('!!')[-1]
        code_label_dictionary[code] = label 

    return code_label_dictionary

In [154]:
'''
Master Function 
'''
def view_acs_codes(json_url, filter_words, blocked_words): 
    
    # Read JSON 
    json_dictionary         = read_census_json(json_url)
    group_dictionary        = return_group_dictionary(json_dictionary)

    # View the contents 
    acs_dataframe          = print_group_dictionary(group_dictionary, filter_words, blocked_words, print_groups = False, print_codes = False)
    
    return json_dictionary, group_dictionary, acs_dataframe 

In [155]:
'''
Parameters
'''
import requests 
import pandas as pd

json_url       = r'https://api.census.gov/data/2022/acs/acs5/variables.json'
filter_words   = ['Family']
blocked_words  = ['(']

In [159]:
'''
Run 
'''
json_dictionary, group_dictionary, acs_dataframe = view_acs_codes(json_url, filter_words, blocked_words)

acs_dataframe

JSON dictionary returned
Group dictionary returned
Group dictionary filtered and passed to dataframe


,Group Codes,Concepts
0,B17022,Ratio of Income to Poverty Level in the Past 1...
1,B17010,Poverty Status in the Past 12 Months of Famili...
2,B23009,Presence of Own Children Under 18 Years by Fam...
3,B17016,Poverty Status in the Past 12 Months of Famili...
4,B11003,Family Type by Presence and Age of Own Childre...
5,B10010,Median Family Income for Families With Grandpa...
6,B17013,Poverty Status in the Past 12 Months of Famili...
7,B11004,Family Type by Presence and Age of Related Chi...
8,B19122,Number of Earners in Family
9,B99193,Allocation of Family Income in the Past 12 Mon...
